# Monkey Breed classifier using Mobilenet

### Loading the Mobilenet model

In [2]:
from tensorflow.keras.applications import MobileNet

img_rows = 224
img_cols = 224

#Re-load the MobileNet model without top 4 layers

MobileNet = MobileNet(weights = "imagenet",
                      include_top = False,
                      input_shape = (img_rows, img_cols, 3))

#Here freeze the last 4 layers
#Layers are set to trainable as True by default

for layer in MobileNet.layers:
    layer.trainable = False
    
#print the trainable Layers
for i, layer in enumerate(MobileNet.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)


0 InputLayer False
1 Conv2D False
2 BatchNormalization False
3 ReLU False
4 DepthwiseConv2D False
5 BatchNormalization False
6 ReLU False
7 Conv2D False
8 BatchNormalization False
9 ReLU False
10 ZeroPadding2D False
11 DepthwiseConv2D False
12 BatchNormalization False
13 ReLU False
14 Conv2D False
15 BatchNormalization False
16 ReLU False
17 DepthwiseConv2D False
18 BatchNormalization False
19 ReLU False
20 Conv2D False
21 BatchNormalization False
22 ReLU False
23 ZeroPadding2D False
24 DepthwiseConv2D False
25 BatchNormalization False
26 ReLU False
27 Conv2D False
28 BatchNormalization False
29 ReLU False
30 DepthwiseConv2D False
31 BatchNormalization False
32 ReLU False
33 Conv2D False
34 BatchNormalization False
35 ReLU False
36 ZeroPadding2D False
37 DepthwiseConv2D False
38 BatchNormalization False
39 ReLU False
40 Conv2D False
41 BatchNormalization False
42 ReLU False
43 DepthwiseConv2D False
44 BatchNormalization False
45 ReLU False
46 Conv2D False
47 BatchNormalization False
48

# Create a function that return the FC Head

In [3]:
def addTopmodel(bottom_model, num_classes):
    
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation = "relu")(top_model)
    top_model = Dense(1024,activation = "relu")(top_model)
    top_model = Dense(512,activation = "relu")(top_model)
    top_model = Dense(num_classes,activation = "softmax")(top_model)
    
    return top_model

# Add FC Head onto MobileNet model

In [4]:
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, ZeroPadding2D
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import Model


In [5]:
num_classes = 10

FC_Head  = addTopmodel(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head)

print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

# Loading Monkey Breed Dataset

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_dir = "./Classifier/monkey_breed/train"
validation_data_dir = "./Classifier/monkey_breed/validation"

#Lets do some Data Augmentation

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   horizontal_flip = True,
                                   fill_mode = "nearest")



validation_datagen = ImageDataGenerator(rescale = 1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (img_rows, img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = "categorical")

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                              target_size = (img_rows, img_cols),
                                                              batch_size = batch_size,
                                                              class_mode = "categorical")

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


# Training Our Model

In [7]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [8]:
checkpoint = ModelCheckpoint("monkey_breed_mobileNet.h5",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1)

earlystop = EarlyStopping(monitor = "val_loss",
                          min_delta = 0,
                          patience = 3,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [checkpoint, earlystop]

model.compile(loss = "categorical_crossentropy",
              optimizer = RMSprop(lr = 0.001),
              metrics = ["accuracy"])

train_samples_nb = 1098
validation_samples_nb = 272

epochs = 5
batch_size = 32

history = model.fit_generator(train_generator,
                              steps_per_epoch = train_samples_nb//batch_size,
                              epochs = epochs,
                              callbacks = callbacks,
                              validation_data = validation_generator)

/home/robin/anaconda3/envs/robin/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
/home/robin/anaconda3/envs/robin/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
2021-09-24 01:45:45.417823: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 19267584 exceeds 10% of free system memory.
2021-09-24 01:45:47.882419: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-24 01:45:48.036943: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2194775000 Hz


Epoch 1/5


2021-09-24 01:45:52.936854: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 19267584 exceeds 10% of free system memory.
2021-09-24 01:45:52.986194: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 51380224 exceeds 10% of free system memory.
2021-09-24 01:45:53.605931: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 51380224 exceeds 10% of free system memory.
2021-09-24 01:45:53.780221: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 51380224 exceeds 10% of free system memory.


34/34 [==============================] - 99s 3s/step - loss: 2.3908 - accuracy: 0.3790 - val_loss: 0.8299 - val_accuracy: 0.6691

Epoch 00001: val_loss improved from inf to 0.82988, saving model to monkey_breed_mobileNet.h5
Epoch 2/5
34/34 [==============================] - 92s 3s/step - loss: 0.9067 - accuracy: 0.7158 - val_loss: 0.4716 - val_accuracy: 0.8750

Epoch 00002: val_loss improved from 0.82988 to 0.47162, saving model to monkey_breed_mobileNet.h5
Epoch 3/5
34/34 [==============================] - 85s 3s/step - loss: 0.6547 - accuracy: 0.8096 - val_loss: 0.4843 - val_accuracy: 0.8603

Epoch 00003: val_loss did not improve from 0.47162
Epoch 4/5
34/34 [==============================] - 73s 2s/step - loss: 0.6242 - accuracy: 0.8377 - val_loss: 0.1445 - val_accuracy: 0.9559

Epoch 00004: val_loss improved from 0.47162 to 0.14445, saving model to monkey_breed_mobileNet.h5
Epoch 5/5
34/34 [==============================] - 71s 2s/step - loss: 0.3409 - accuracy: 0.9081 - val_loss: 

In [9]:
print(history.history)

{'loss': [2.3908064365386963, 0.9067190885543823, 0.6547368168830872, 0.6242305040359497, 0.3408568501472473], 'accuracy': [0.3789868652820587, 0.7157598733901978, 0.809568464756012, 0.8377110958099365, 0.908067524433136], 'val_loss': [0.8298754692077637, 0.47162431478500366, 0.48426902294158936, 0.14445066452026367, 0.13497108221054077], 'val_accuracy': [0.6691176295280457, 0.875, 0.8602941036224365, 0.9558823704719543, 0.9595588445663452]}


# Save the History

In [10]:
import pickle

pickle_out = open("Monkey_breed_history.pickle","wb")
pickle.dump(history.history, pickle_out)
pickle_out.close()

In [11]:
pickle_in = open("Monkey_breed_history.pickle","rb")
save_history = pickle.load(pickle_in)
print(save_history)

{'loss': [2.3908064365386963, 0.9067190885543823, 0.6547368168830872, 0.6242305040359497, 0.3408568501472473], 'accuracy': [0.3789868652820587, 0.7157598733901978, 0.809568464756012, 0.8377110958099365, 0.908067524433136], 'val_loss': [0.8298754692077637, 0.47162431478500366, 0.48426902294158936, 0.14445066452026367, 0.13497108221054077], 'val_accuracy': [0.6691176295280457, 0.875, 0.8602941036224365, 0.9558823704719543, 0.9595588445663452]}


In [2]:
##loading the model

from tensorflow.keras.models import load_model

clf = load_model("monkey_breed_mobileNet.h5")

# Testing Classifier on Test image

In [4]:
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [5]:
monkey_breed_dict = {"[0]": "manted_howler",
                     "[1]": "patas_monkey",
                     "[2]": "bald_uakari",
                     "[3]": "japanese_macaque",
                     "[4]": "pygmy_marmoset",
                     "[5]": "white_headed_capuchin",
                     "[6]": "silvary_marmoset",
                     "[7]": "common_squirrel_monkey",
                     "[8]": "black_headed_night_monkey",
                     "[9]": "nilgiri_langur"}

#this dict is for showing the true result from validation folder
monkey_breed_dict_n = {"n0": "manted_howler",
                     "n1": "patas_monkey",
                     "n2": "bald_uakari",
                     "n3": "japanese_macaque",
                     "n4": "pygmy_marmoset",
                     "n5": "white_headed_capuchin",
                     "n6": "silvary_marmoset",
                     "n7": "common_squirrel_monkey",
                     "n8": "black_headed_night_monkey",
                     "n9": "nilgiri_langur"}


def draw_test(name, pred, im):
    monkey = monkey_breed_dict[str(pred)]
    
    BLACK = [0,0,0]
    expanded_img = cv2.copyMakeBorder(im, 80, 0,0, 100, cv2.BORDER_CONSTANT, value = BLACK)
    cv2.putText(expanded_img, monkey, (20,60), cv2.FONT_HERSHEY_PLAIN,1,(0,0,255), 2)
    cv2.imshow(name, expanded_img)
    

def getRandomImg(path):
    """loads a random image from random folder in our test path"""
    
    folder = list(filter(lambda x:os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    ran_directory = np.random.randint(0,len(folder))
    path_class = folder[ran_directory]
    #print(path_class)
    print("Class - " + monkey_breed_dict_n[str(path_class)])
    file_path = path + path_class
    file_name = [f for f in listdir(file_path) if isfile(join(file_path,f))]
    random_file_index = np.random.randint(0, len(file_name))
    image_name = file_name[random_file_index]
    
    return cv2.imread(file_path+"/"+image_name)

for i in range(0,10):
    input_im = getRandomImg("./Classifier/monkey_breed/validation/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224,224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255
    input_im = input_im.reshape(1,224,224,3)
    
    #Get prediction 
    res = np.argmax(clf.predict(input_im, 1, verbose = 0), axis = 1)
    
    
    #show image with predicted class
    draw_test("prediction", res, input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

Class - pygmy_marmoset


2021-09-26 01:37:36.264422: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-09-26 01:37:36.281835: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2194805000 Hz


Class - silvary_marmoset
Class - patas_monkey
Class - silvary_marmoset
Class - manted_howler
Class - nilgiri_langur
Class - common_squirrel_monkey
Class - nilgiri_langur
Class - pygmy_marmoset
Class - black_headed_night_monkey
